<a href="https://colab.research.google.com/github/D-Yadav/NLP/blob/main/Fake_News_Classifier_using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier using Bidirectional LSTM

In [1]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd


In [4]:
df=pd.read_csv('/train.csv',engine='python',error_bad_lines=False)

<ipython-input-4-5c62e5a077ef>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('/train.csv',engine='python',error_bad_lines=False)
Skipping line 810: unexpected end of data


In [5]:
df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id         0
title     21
author    85
text       2
label      0
dtype: int64

In [7]:
df.shape

(808, 5)

In [8]:
df = df.dropna()

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
# get the independent features
X = df.drop('label', axis=1)

In [11]:
# dependent features
y = df['label']

In [12]:
# Check whether dataset is balanced or not
y.value_counts()

0    402
1    300
Name: label, dtype: int64

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.12.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [16]:
## vocabulary size
voc_size=1000

In [17]:
messages = X.copy()

In [18]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
802,802,Trump Team’s Links to Russia Crisscross in Was...,Scott Shane and Andrew E. Kramer,"WASHINGTON — During the 2016 campaign, Dona..."
803,803,North Korean Is Arrested in Killing of Kim Jon...,Richard C. Paddock and Rick Gladstone,"KUALA LUMPUR, Malaysia — A man from North K..."
804,804,SURGEONS ADMIT THAT MAMMOGRAPHY IS OUTDATED AN...,newsguy76,Home › HEALTH › SURGEONS ADMIT THAT MAMMOGRAPH...
806,806,WAR ON THE STREETS OF PARIS: Armed migrants fi...,wmw_admin,"By wmw_admin on November 3, 2016 Nick Gutterid..."


In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
messages.reset_index(inplace=True)

In [22]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[142, 815, 799, 728, 602, 636, 745, 395, 428, 95],
 [254, 610, 184, 476, 277, 608, 772],
 [190, 616, 237, 804],
 [902, 917, 572, 532, 819, 43],
 [497, 277, 376, 589, 471, 313, 277, 199, 373, 436],
 [62, 716, 210, 876, 183, 63, 259, 781, 198, 788, 311, 589, 845, 863, 772],
 [523, 699, 211, 806, 859, 972, 112, 596, 321, 407, 522],
 [262, 638, 104, 972, 786, 852, 63, 704, 321, 407, 522],
 [146, 265, 44, 612, 733, 920, 59, 459, 63, 763],
 [391, 428, 262, 975, 492, 329, 84, 976],
 [472, 298, 114, 384, 532, 797, 776, 301, 618, 806, 417],
 [532, 925, 602, 920, 63, 786],
 [840, 560, 716, 100, 69, 316, 918, 720, 380],
 [556, 619, 950, 744, 938, 926, 840, 321, 407, 522],
 [507, 667, 476, 290, 230, 321, 407, 522],
 [174, 151, 699, 525, 786, 250, 919, 639, 471, 378],
 [460, 154, 610],
 [55, 412, 682, 529, 63, 645, 848, 772],
 [822, 829, 184, 419, 619, 603, 77, 915, 313],
 [192, 548, 63, 941, 65, 772],
 [267, 764, 810, 235, 836, 77, 259, 98, 646, 321, 407, 522],
 [841, 201, 209, 478, 257, 282, 826

### Embedding Representation

In [26]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[  0,   0,   0, ..., 395, 428,  95],
       [  0,   0,   0, ..., 277, 608, 772],
       [  0,   0,   0, ..., 616, 237, 804],
       ...,
       [  0,   0,   0, ..., 312, 437, 273],
       [  0,   0,   0, ...,  92, 806, 299],
       [  0,   0,   0, ..., 716,  63, 630]], dtype=int32)

In [30]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            40000     
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 153,001
Trainable params: 153,001
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final = np.array(y)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [34]:
## Model Training
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=32)

Epoch 1/10
15/15 [==============================] - 13s 245ms/step - loss: 0.6852 - accuracy: 0.5298 - val_loss: 0.6562 - val_accuracy: 0.6164
Epoch 2/10
15/15 [==============================] - 2s 122ms/step - loss: 0.6374 - accuracy: 0.6851 - val_loss: 0.5663 - val_accuracy: 0.8276
Epoch 3/10
15/15 [==============================] - 1s 92ms/step - loss: 0.3464 - accuracy: 0.8660 - val_loss: 0.4450 - val_accuracy: 0.8664
Epoch 4/10
15/15 [==============================] - 2s 124ms/step - loss: 0.0852 - accuracy: 0.9809 - val_loss: 0.5474 - val_accuracy: 0.8750
Epoch 5/10
15/15 [==============================] - 1s 103ms/step - loss: 0.0440 - accuracy: 0.9915 - val_loss: 0.4130 - val_accuracy: 0.9009
Epoch 6/10
15/15 [==============================] - 1s 68ms/step - loss: 0.0470 - accuracy: 0.9830 - val_loss: 0.5632 - val_accuracy: 0.8922
Epoch 7/10
15/15 [==============================] - 1s 39ms/step - loss: 0.0788 - accuracy: 0.9872 - val_loss: 0.7574 - val_accuracy: 0.8448
Epoch 8/

In [35]:
y_pred=model.predict(X_test)

8/8 [==============================] - 1s 3ms/step


In [40]:
y_pred = np.where(y_pred>=0.5,1,0)

In [44]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[134,   9],
       [ 21,  68]])

In [45]:
print(accuracy_score(y_test,y_pred))

0.8706896551724138
